# deepOrganoid: Brightfeld viability assay for matrix embedded organoids

Select the first box and press SHIFT+ENTER. This will run that block and advance to the next

In [29]:
from __future__ import print_function
import os

import sys
import time
import yaml
import pickle
import traceback
import pandas as pd
import numpy as np
import scipy.misc
import scipy.io.wavfile
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from importlib.machinery import SourceFileLoader
from multiprocessing import cpu_count
from matplotlib.pyplot import imread
from PIL import Image
print("All libraries have been loaded")

All libraries have been loaded


**The subsequent box should match the following:**<br />
Tensorflow: 2.1.0<br />
nvcc: NVIDIA (R) Cuda compiler driver<br />
Copyright (c) 2005-2020 NVIDIA Corporation<br />
Built on Thu_Jun_11_22:26:48_Pacific_Daylight_Time_2020<br />
Cuda compilation tools, release 11.0, V11.0.194<br />
Build cuda_11.0_bu.relgpu_drvr445TC445_37.28540450_0<br />

In [30]:
print("Tensorflow: ",tf.__version__)
!nvcc --version

Tensorflow:  2.1.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Thu_Jun_11_22:26:48_Pacific_Daylight_Time_2020
Cuda compilation tools, release 11.0, V11.0.194
Build cuda_11.0_bu.relgpu_drvr445TC445_37.28540450_0


# User inputs

In [31]:
#Must download models first
#Options: AutoML, Lite_vr1, Lite_vr2, Lite_vr3, Final. Refer to text for differences
rel_model_path = 'Models/AutoML'

#Base directory with the images
#WILL NEED TO BE EDITED, For windows remember to use either \\ or / as path deliminters 
rel_Input_path = 'Example_data' 

#The enumerated GPU device you wish to use 0,1,nGPU
cuda_device = 0

# deepOrganoid code

In [32]:
Base = os.getcwd()
Base = 'Q:/CTCR_DeepHTS/deepOrganoid/'
model_path = os.path.join(Base,rel_model_path)
Input_path = os.path.join(Base,rel_Input_path)

if os.path.exists(model_path):
    print("Found: ",model_path)
    os.chdir(model_path)
    model_name = os.path.basename(model_path)
else:
    print('Model path not found')
    
if os.path.exists(Input_path):
    print("Found: ",Input_path)
else:
    print('Image directory not found')

Found:  Q:/CTCR_DeepHTS/deepOrganoid/Models/AutoML
Found:  Q:/CTCR_DeepHTS/deepOrganoid/Example_data


In [33]:
os.environ["MKL_NUM_THREADS"] = str(cpu_count())
os.environ["MKL_NUM_THREADS"]

'16'

# Import inference functions

In [39]:
def doResize(options):
    resize = None
    if options and 'Resize' in options and options['Resize'] == True:
        resize = (int(options['Width']), int(options['Height']))
    return resize

def col_pre_process(data, options):
    if len(options.keys()) == 0:
        return data
    else:
        if "Scaling" in options and float(options["Scaling"]) != 0 and float(options["Scaling"]) != 1:
            data = data / float(options["Scaling"])

        if 'Normalization' in options and options['Normalization'] == True:
            mean = np.mean(data)
            std = np.std(data)
            data = data - mean
            data = data / std
            return data
        return data

def process_test_input(base_dir, test_raw, data_mapping):
    test_data = []
    le = None

    from tensorflow.keras import backend as K
    if K.backend() == 'theano' or K.backend() == 'mxnet':
        K.set_image_data_format('channels_first')
    else:
        K.set_image_data_format('channels_last')

    # determine the shape of the data to feed into the network
    for i in range(len(data_mapping['inputs'])):
        inp_port = data_mapping['inputs']['InputPort' + str(i)]
        if inp_port['details'][0]['type'] == 'Image':
            col_name = inp_port['details'][0]['name']
            if 'options' in inp_port['details'][0]:
                options = inp_port['details'][0]['options']
            else:
                options = {}

            resize = doResize(options)
            img = imread(test_raw[col_name][0])
            input_shape = img.shape

            num_channels = 1
            if resize:
                width, height = resize
                if len(input_shape) == 3:
                    num_channels = 3
            else:
                if len(input_shape) == 2:
                    width, height = input_shape
                else:
                    width, height, num_channels = input_shape
            test_data.append(np.ndarray((len(test_raw),) +
                                        (num_channels, width, height), dtype=np.float32))

            for j, filename in enumerate(test_raw[col_name]):
                img = imread(filename)
                if resize:
                    img = np.array(Image.fromarray(img.astype(np.uint8)).resize(resize))
                if num_channels != 1:
                    img = np.transpose(img, (2, 0, 1))
                test_data[i][j] = img

            if K.image_data_format() == 'channels_last':
                test_data[i] = np.transpose(test_data[i], (0, 2, 3, 1))

            test_data[i] = col_pre_process(test_data[i], options)

    # assuming single output, generate labelEncoder
    out_port = data_mapping['outputs']['OutputPort0']
    if out_port['details'][0]['type'] == 'Categorical':
        le = LabelEncoder()
        le.fit(out_port['details'][0]['categories'])
    return test_data, le

def customPredict(test_data, config, modelFile):
    res = None
    loss_func = config['params']['loss_func']
    if 'is_custom_loss' in config['params']:
        isCustomLoss = config['params']['is_custom_loss']
    else:
        isCustomLoss = False
    if isCustomLoss:
        customLoss = SourceFileLoader(
            "customLoss", 'customLoss.py').load_module()
        loss_function = eval('customLoss.' + loss_func)
        mod = load_model(modelFile, custom_objects={loss_func: loss_function})
    else:
        mod = load_model(modelFile)
    
    if os.environ.get("GPU_ENABLED", "0") == "1":
        mod.compile (loss='categorical_crossentropy', optimizer='adam', context=["GPU("+str(cuda_device)+")"])
    with tf.device("GPU:"+str(cuda_device)):
        ress = mod.predict(test_data,batch_size=4)
    return ress

def test_model(input_file):
    try:
        if os.path.exists('model.h5') and os.path.exists('mapping.pkl'):
            with open('mapping.pkl', 'rb') as f:
                data_mapping = pickle.load(f)
            test_raw = pd.read_csv(input_file)
            test_data, le = process_test_input(
                os.path.dirname(input_file), test_raw, data_mapping)
            currentDir = os.getcwd()
            with open('config.yaml', 'r') as f:
                config = yaml.load(f, Loader=yaml.Loader)
                models = []
                models.append(currentDir + '/model.h5')

            result = np.array([])
            for modelFile in models:
                res = customPredict(test_data, config, modelFile)
                if result.size != 0:
                    result = res + result
                else:
                    result = res
            res = result / len(models)

            out_type = data_mapping['outputs']['OutputPort0']['details'][0]['type']

            num_samples = len(test_raw)
            if num_samples != 0:
                out_dir = "./"
                if not os.path.exists(out_dir + "output/"):
                    os.makedirs(out_dir + "output/")
                if out_type == "Numpy":
                    if not os.path.exists(out_dir + "output/"):
                        os.makedirs(out_dir + "output/")
                    temp = np.ndarray((res.shape[0],), dtype=np.object_)
                    for i in range(res.shape[0]):
                        filename = "./output/" + str(i) + ".npy"
                        np.save(out_dir + filename, res[i])
                        temp[i] = filename
                    test_raw['predictions'] = temp
                test_raw.to_csv('test_result.csv', index=False)
        else:
            print('model or data mapping does not exist... try downloading again!')
    except Exception as e:
        print("aborting due to exception... Please check input file format!")
        traceback.print_exc()

# Run classification by batch (sub-directory)

In [40]:
import shutil
import timeit
from tensorflow.keras.models import load_model

if os.path.exists('model.h5') and os.path.exists('mapping.pkl'):
    with open('mapping.pkl', 'rb') as f:
        data_mapping = pickle.load(f)
    
    for i in range(len(data_mapping['inputs'])):
        inp_port = data_mapping['inputs']['InputPort' + str(i)]
        if inp_port['details'][0]['type'] == 'Image':
            Column_name = inp_port['details'][0]['name']

df = pd.DataFrame()
for all_proj in os.listdir(Input_path):
    if '_Proj' in all_proj: ##These conditional statments may need to be edited for new applications
        start_time = timeit.default_timer()
        active_dir = os.path.join(Input_path,all_proj)
        
        name = model_name+".csv"
        skip = os.path.isfile(os.path.join(active_dir,name))
        
        if skip:
            print("Skipping",active_dir)
        else:
            print("Building input: ",active_dir)
            df = pd.DataFrame()
            for root, dirs, files in os.walk(active_dir):
                for name in files:
                    if '.tif' in name and 'ch01' in name:
                        active = pd.Series(os.path.join(root, name),name = Column_name)
                        df = df.append(active,ignore_index = True)
            
            df = df.rename(columns={0:Column_name})
            DLS_input = os.path.join(active_dir,'test.csv')
            df.to_csv(DLS_input, index=False)
            
            print("Running deepOrganoid")
            test_model(DLS_input)
            name = model_name+".csv"
            shutil.move(os.path.join(model_path,"test_result.csv"),os.path.join(active_dir,name))
            os.remove(DLS_input)
            
            elapsed = timeit.default_timer() - start_time
            print("Data has been saved at: ",all_proj,"\nTime to completion: ",elapsed/60,"minutes")

print("All tasks completed, have yourself a drink")

Building input:  Q:/CTCR_DeepHTS/deepOrganoid/Example_data\2019-04-29_Proj
Running deepOrganoid
Data has been saved at:  2019-04-29_Proj 
Time to completion:  0.786632441666734 minutes
All tasks completed, have yourself a drink


# Release/Clean GPU VRAM

In [9]:
#Clear GPU RAM
from numba import cuda
for index, device in enumerate(cuda.gpus):
    cuda.select_device(index)
    device = cuda.get_current_device()
    device.reset()

# Scratch space